In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
import pandas as pd
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence

In [2]:
# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

/home/mojo/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


/home/mojo/.local/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:53: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(
Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


OSError: Can't load feature extractor for 'facebook/wav2vec2-large-960h-lv60-self'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'facebook/wav2vec2-large-960h-lv60-self' is the correct path to a directory containing a preprocessor_config.json file

In [ ]:
# Read the list of audio file paths
file_path = 'path_test.txt'
with open(file_path, 'r') as file:
    audio_paths = file.readlines()

In [ ]:
# Duration to extract (30 seconds)
duration = 5.0

In [ ]:
# List to store the padded sequences for each layer
all_padded_sequences = [[] for _ in range(model.config.num_hidden_layers + 1)]

In [ ]:
# Process each audio file
for audio_path in tqdm(audio_paths):
    audio_path = audio_path.strip()
    
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_path)
    
    # Ensure the audio is at least 30 seconds long
    if len(audio_input) < duration * sample_rate:
        continue
    
    # Extract the first 30 seconds of audio
    audio_input = audio_input[:int(duration * sample_rate)]
    
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values
    
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
    
    # Get the features from all layers, including the last hidden state
    all_hidden_states = outputs.hidden_states
    
    # Store the features for each layer
    for i, layer_features in enumerate(all_hidden_states):
        all_padded_sequences[i].append(layer_features.squeeze(0))

In [ ]:
# Determine the maximum sequence length
max_length = max([seq.size(0) for layer in all_padded_sequences for seq in layer])

# Pad each sequence to the maximum length
padded_sequences = [pad_sequence(layer, batch_first=True, padding_value=0) for layer in all_padded_sequences]

In [ ]:
# Save each layer's padded features to a CSV file
for i, layer_features in enumerate(padded_sequences):
    # Flatten the features
    layer_flat = layer_features.view(-1, layer_features.size(-1)).numpy()
    
    # Convert to DataFrame
    df_layer = pd.DataFrame(layer_flat)
    
    # Save to CSV
    df_layer.to_csv(f"layer_{i}_features_30s.csv", index=False)